# Initial data exploration

In [1]:
import glob
import re
import sys
import random
import matplotlib
import matplotlib.pyplot as plt
from gensim import models, corpora


In [2]:
import nltk
nltk.download('cess_esp')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords


[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Unzipping corpora/cess_esp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


## Utility functions


In [3]:
# Random seed
random.seed(498)

def get_data(folder="../captions/"):
    line_re = re.compile(r"^(?:\d+(?:\.\d+)?\s+)+(.+)$")
    docs = []
    for file in glob.glob(folder + "*_es.txt"):
        file_lines = []
        with open(file) as fp:
            for line in fp:
                m = line_re.match(line)
                file_lines.append(m.group(1))
        docs.append(' '.join(file_lines))
    return docs


def get_tokenized_data(data):
      return [word_tokenize(text.lower()) for text in data]

def get_word_list(tokenized_text):
    word_dict = dict()
    for tokens in tokenized_text:
        for token in tokens:
            if token not in word_dict:
                word_dict[token] = 0
            word_dict[token] += 1
    return list(word_dict.items())


## Load data

In [4]:
data = get_data()
print("Number of transcriptions", len(data))

Number of transcriptions 105


### Sample transcription

In [5]:
print(data[random.randint(0, len(data))])

bueno pues muy buenos días tal cómo que anunció el presidente ahorita a las de la mañana el secretario de comunicaciones de transportes y el subsecretario darán a conocer los resultados de todos los estudios que se realizaron desde el punto de vista técnico con respecto al accidente de helicóptero con el lamentable saldo de dos de vidas perdidas entonces sin más le cedo la palabra al señor secretario muchas gracias muy buenos días muchas gracias por su atención y por su interés como fue dicho desde un principio por el señor presidente de la república las investigaciones en relación con el lamentable accidente el helicóptero en que perdió la vida la señora gobernadora del estado de puebla su esposo y celador un acompañante y los tripulantes será dado a conocer periódicamente en detalle con toda transparencia y con toda claridad no vamos a ocultar en este caso ningún punto en relación con las investigaciones el ingeniero carlos morán moguel subsecretario del transporte que ha lanzado a c

### Word count

In [6]:
tokenized_text = [word_tokenize(text.lower()) for text in data]

In [7]:
word_list = sorted(get_word_list(tokenized_text), key = lambda x: -x[1])

In [8]:
print('Unique word count', len(word_list))
print('Top 10 words', word_list[0:10])

Unique word count 24109
Top 10 words [('de', 38355), ('que', 28120), ('la', 23037), ('a', 20307), ('y', 18451), ('el', 18425), ('en', 16753), ('se', 13345), ('los', 11310), ('no', 10679)]


#### Remove irrelevant words


In [9]:
STOPWORDS = stopwords.words('spanish')

def clean_text(text):
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'

    tokenized_text = word_tokenize(text.lower())
    all_nouns = [word for (word, pos) in pos_tag(tokenized_text) if is_noun_adj(pos)]

    cleaned_text = [t for t in all_nouns if t not in STOPWORDS and re.match(r"[a-zA-Z\-][a-zA-Z\-]{2,}", t)]
    return cleaned_text


tokenized_text_no_stop_words = [clean_text(text) for text in data]
word_list = sorted(get_word_list(tokenized_text_no_stop_words), key = lambda x: -x[1])
print('Unique word count', len(word_list))
print('Top 10 words', word_list[0:10])

Unique word count 18924
Top 10 words [('entonces', 2006), ('pues', 1503), ('presidente', 1425), ('gobierno', 1223), ('ser', 1078), ('hacer', 920), ('bueno', 918), ('mil', 838), ('tener', 830), ('caso', 785)]


In [10]:
words = [word[0] for word in word_list]

In [12]:
with open('./spanish_words.txt', 'w') as f:
    for item in word_list:
        f.write("%s,%d\n" % (item[0], item[1]))